In [1]:
import itertools
import operator
import pickle
import re
from abc import *
from copy import deepcopy
from operator import itemgetter
from typing import *
import numpy as np
import scipy as sp
import networkx as nx
import mne
import time
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.neighbors._dist_metrics import DistanceMetric
from sklearn.utils import shuffle
from typing import List, Tuple
from sklearn.model_selection import train_test_split

from nodestimation.learning.connectoming import make_connectome
from nodestimation.learning.estimation import collect_statistic, \
    compute_importance, collect_cross_statistic, make_selection_map, \
    select, separate_datasets, selected_statistic, choose_best, selected_data, make_feature_selection
from nodestimation.learning.informativeness import CrossInformativeness, Informativeness, SubjectsInformativeness, \
    NodesInformativeness
from nodestimation.learning.networking import sparse_graph, graph_to_hemispheres, hemispheres_division_modularity, \
    metric_for_hemispheres
from nodestimation.processing.features import prepare_features
from nodestimation.project import find_subject_dir, conditions_unique_code
from nodestimation.pipeline import pipeline
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
import nibabel
import nilearn.plotting as nplt
from nodestimation.project.actions import read
import nodestimation as nd
from nodestimation.learning.modification import append_series, promote
import nodestimation.learning.modification as lmd
from nodestimation.project.subject import Subject
from sklearn.preprocessing import *
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.feature_selection import SelectFromModel
from sklearn.cluster import KMeans
from sklearn.neural_network import MLPClassifier
import matplotlib as mpl
from nodestimation.learning.selection import SubjectsStatistic, Wilcoxon, Mannwhitneyu, Test
from scipy.stats import wilcoxon
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import SGDClassifier
from scipy.fftpack import fftfreq, irfft, rfft
from scipy.fftpack import fftfreq, irfft, rfft

ENGEL1 = [
    'B1C2',
    'B1R1',
    'G1R1',
    'G1V2',
    'J1T2',
    'K1V1',
    'L1P1',
    'M1G2',
    'M1N2',
    'O1O2',
    'R1D2',
    'S1A2',
    'S1B1',
    'S1H1',
    'S1U3'
]
ENGEL2 = [
    'L2M1',
    'M2S2',
    'N2K2',
    'P1H2'
]
ENGEL3 = [
    'N3S2',
    'S3R1'
]
ENGEL4 = [
    'K4L2'
]
REJECTED = [
    'S1U3',
    'P1H2'
]

AGE = {
    'B1C2': 28.0,
    'B1R1': 16.0,
    'G1R1': 23.0,
    'G1V2': 5.0,
    'J1T2': 8.0,
    'K1V1': 10.0,
    'K4L2': 14.0,
    'L1P1': 16.0,
    'L2M1': 20.0,
    'M1G2': 8.0,
    'M1N2': 7.0,
    'M2S2': 18.0,
    'N2K2': 30.0,
    'N3S2': 10.0,
    'O1O2': 18.0,
    'R1D2': 6.5,
    'P1H2': 7.0,
    'S1A2': 12.0,
    'S1B1': 17.0,
    'S1H1': 28.0,
    'S3R1': 19.0,
    'S1U3': 15.0,
}

SEX = {
    'B1C2': 'f',
    'B1R1': 'm',
    'G1R1': 'f',
    'G1V2': 'm',
    'J1T2': 'f',
    'K1V1': 'f',
    'K4L2': 'f',
    'L1P1': 'f',
    'L2M1': 'f',
    'M1G2': 'm',
    'M1N2': 'm',
    'M2S2': 'm',
    'N2K2': 'm',
    'N3S2': 'm',
    'O1O2': 'f',
    'R1D2': 'f',
    'P1H2': 'm',
    'S1A2': 'm',
    'S1B1': 'm',
    'S1H1': 'm',
    'S3R1': 'm',
    'S1U3': 'f',
}

SUBJECTS = pipeline(
    methods=['wpli', 'envelope', 'coh', 'imcoh', 'plv', 'ciplv', 'ppc', 'pli', 'pli2_unbiased', 'wpli2_debiased'],
    freq_bands=(4, 8),
    centrality_metrics=['eigen', 'between', 'degree', 'info']
    )

CONNECTOMES_KIND = 'initial'

/home/user/.local/lib/python3.9/site-packages/nilearn/datasets/__init__.py:86: FutureWarning: Fetchers from the nilearn.datasets module will be updated in version 0.9 to return python strings instead of bytes and Pandas dataframes instead of Numpy arrays.
  warn("Fetchers from the nilearn.datasets module will be "


All computation has been already done, loading of the existing file with the solution...


In [2]:
subjects = SUBJECTS.copy()

# DATASETS = {
#     subject.name: {
#         freq: {
#             method: make_connectome(subject, freq, method, CONNECTOMES_KIND, threshold=1)
#            for method in subject.connectomes[freq]
#         }
#         for freq in subject.connectomes
#     }
#     for subject in subjects
# }
# print('Connectomes are prepared')

GRAPHS = [
    metric_for_hemispheres(subjects, nx.algorithms.cluster.transitivity),
    metric_for_hemispheres(subjects, nx.algorithms.smetric.s_metric, normalized=False),
    metric_for_hemispheres(subjects, nx.algorithms.global_efficiency),
]
print('Graphs are prepared')

Graphs are prepared


In [ ]:
print('Processing centrality...')

datasets = DATASETS.copy()

ignored_methods = [
#     'wpli',
#     'envelope',
#     'coh',
#     'imcoh',
#     'plv',
#     'ciplv',
#     'ppc',
#     'pli',
#     'pli2_unbiased',
#     'wpli2_debiased'
]

close = dict()
between = dict()
eigen = dict()
degree = dict()
info = dict()
harmony = dict()
katz = dict()
percolation = dict()

total_ignored_subjects = 0

for subject_name in datasets:
    is_corrupted = False
    close.update({subject_name: dict()})
    between.update({subject_name: dict()})
    eigen.update({subject_name: dict()})
    degree.update({subject_name: dict()})
    info.update({subject_name: dict()})
    harmony.update({subject_name: dict()})
    katz.update({subject_name: dict()})
    percolation.update({subject_name: dict()})
    for freq in datasets[subject_name]:
        if is_corrupted:
            break
        for method in datasets[subject_name][freq]:

            if method in ignored_methods:
                continue

            print(subject_name, freq, method)
            label_names = datasets[subject_name][freq][method].columns

            if nx.is_connected(
                    nx.convert_matrix.from_numpy_array(
                        datasets[subject_name][freq][method].to_numpy()
                    )
            ):
                arr  = datasets[subject_name][freq][method].to_numpy()
            else:
                try:
                    corrupted_subject = [subject for subject in subjects if subject.name == subject_name][0]
                    subjects.remove(corrupted_subject)
                except IndexError:
                    pass
                total_ignored_subjects += 1
                print(f'Graph is not connected: {subject_name}, {freq}, {method}.\n'
                      f'This subject will be ignored.\n'
                      f'Total of ignored subjects: {total_ignored_subjects}\n'
                      f'Total of spared subjects: {len(subjects)}\n')
                is_corrupted = True
                break
                # raise ValueError(f'Graph is not connected: {subject_name}, {freq}, {method}')
            # if matrix has negative values, shift it to make minimal value at least 0
            if arr.min().min() < 0:
                print(f'\tShifting connectome for {method} at {freq}')
                for i in range(arr.shape[0]):
                    for j in range(arr.shape[1]):
                        if i != j: arr[i, j] -= arr.min().min()
            G = nx.convert_matrix.from_numpy_array(arr)
            mapping = {node: label_name for node, label_name in zip(G, label_names)}
            max = np.max(
                np.array([
                    np.sum(row)
                    for row in arr
                ])
            )
            min = np.min(
                np.array([
                    np.sum(row)
                    for row in arr
                ])
            )
            states = {node: (np.sum(row) - min)/(max - min) for node, row in zip(label_names, arr)}
            G_num = G.copy()
            G = nx.relabel_nodes(G, mapping)
            if CONNECTOMES_KIND == 'initial' or CONNECTOMES_KIND == 'inverse' or CONNECTOMES_KIND == 'spatial':
                G_sup = nx.convert_matrix.from_numpy_array(
                        lmd.suppress(
                            datasets[subject_name][freq][method],
                            trigger= arr.mean().mean(),
                            optimal=0
                        ).to_numpy()
                    )
                G_sup = nx.relabel_nodes(G_sup, mapping)
            else:
                G_sup = G
            for place, data in zip(
                [
                    close[subject_name],
                    between[subject_name],
                    eigen[subject_name],
                    info[subject_name],
                    degree[subject_name],
                    harmony[subject_name],
                    katz[subject_name],
                    percolation[subject_name]
                ],[
                        nx.closeness_centrality(G, distance='weight'),
                        nx.betweenness_centrality(G_sup, weight='weight'),
                        nx.eigenvector_centrality_numpy(G, weight='weight'),
                        nx.information_centrality(G, weight='weight'),
                        dict(G.degree(weight='weight')),
                        nx.harmonic_centrality(G, distance='weight'),
                        nx.katz_centrality_numpy(G, weight='weight'),
                        nx.algorithms.centrality.percolation_centrality(G_sup, weight='weight', states=states)
                    ]
            ):
                place.update({
                    freq + '_' + method: pd.Series(data)
                })

print('Centrality computations are done')

print('Updating connectomes...')

datasets_centrality = dict()

for subject in subjects:
    datasets_centrality.update({subject.name: dict()})
    datasets_centrality[subject.name].update({
        'close': pd.DataFrame(close[subject.name]),
        'between': pd.DataFrame(between[subject.name]),
        'eigen': pd.DataFrame(eigen[subject.name]),
        'info': pd.DataFrame(info[subject.name]),
        'degree': pd.DataFrame(degree[subject.name]),
        'harmony': pd.DataFrame(harmony[subject.name]),
        'katz': pd.DataFrame(katz[subject.name]),
        'percolation': pd.DataFrame(percolation[subject.name])
    })
    true = subject.datasets['eigen']['resected']
    datasets_centrality[subject.name]['close'] =\
        datasets_centrality[subject.name]['close'].assign(resected=true)
    datasets_centrality[subject.name]['between'] =\
        datasets_centrality[subject.name]['between'].assign(resected=true)
    datasets_centrality[subject.name]['eigen'] =\
        datasets_centrality[subject.name]['eigen'].assign(resected=true)
    datasets_centrality[subject.name]['degree'] =\
        datasets_centrality[subject.name]['degree'].assign(resected=true)
    datasets_centrality[subject.name]['info'] =\
        datasets_centrality[subject.name]['info'].assign(resected=true)
    datasets_centrality[subject.name]['harmony'] =\
        datasets_centrality[subject.name]['harmony'].assign(resected=true)
    datasets_centrality[subject.name]['katz'] =\
        datasets_centrality[subject.name]['katz'].assign(resected=true)
    datasets_centrality[subject.name]['percolation'] =\
        datasets_centrality[subject.name]['percolation'].assign(resected=true)

for subject in subjects:
    subject.datasets = datasets_centrality[subject.name]

print(f'Connectomes successfully updated. Now each subject has new {CONNECTOMES_KIND} connectomes')

print('Collecting statistics...')

stat1 = SubjectsStatistic(subjects, 'resected', centrality_metric='eigen')
print('Eigencentrality statistics are collected')
stat2 = SubjectsStatistic(subjects, 'resected', centrality_metric='between')
print('Betweenness centrality statistics are collected')
stat3 = SubjectsStatistic(subjects, 'resected', centrality_metric='close')
print('Closeness centrality statistics are collected')
stat4 = SubjectsStatistic(subjects, 'resected', centrality_metric='degree')
print('Degree centrality statistics are collected')
stat5 = SubjectsStatistic(subjects, 'resected', centrality_metric='info')
print('Info centrality statistics are collected')
stat6 = SubjectsStatistic(subjects, 'resected', centrality_metric='harmony')
print('Harmony centrality statistics are collected')
stat7 = SubjectsStatistic(subjects, 'resected', centrality_metric='katz')
print('Katz centrality statistics are collected')
stat8 = SubjectsStatistic(subjects, 'resected', centrality_metric='percolation')
print('Percolation centrality statistics are collected')
print(f'All statistics for {CONNECTOMES_KIND} connectomes are collected')
print('Saving statistics')

path = f'/home/user//Documents/{CONNECTOMES_KIND}_stats.pkl'
pickle.dump(
    (
        stat1,
        stat2,
        stat3,
        stat4,
        stat5,
        stat6,
        stat7,
        stat8,
    ),
    open(
        path,
        'wb'
    )
)

print(f'All statistics are successfully saved at {path = }')

In [ ]:
path = f'/home/user//Documents/{CONNECTOMES_KIND}_stats.pkl'
stats = pickle.load(open(path, 'rb'))



In [ ]:
test1 = stats[0].test(state='reflected')
test2 = stats[1].test(state='reflected')
test3 = stats[2].test(state='reflected')
test4 = stats[3].test(state='reflected')
test5 = stats[4].test(state='reflected')
test6 = stats[5].test(state='reflected')
test7 = stats[6].test(state='reflected')
test8 = stats[7].test(state='reflected')
test1_samples, test2_samples, test3_samples, test4_samples, test5_samples, test6_samples = list(), list(), list(), list(), list(), list()
for feature in test1.result:
    test1_samples.append(test1.result[feature][1])
    test2_samples.append(test2.result[feature][1])
    test3_samples.append(test3.result[feature][1])
    test4_samples.append(test4.result[feature][1])
    test5_samples.append(test5.result[feature][1])
    test6_samples.append(test6.result[feature][1])

test_samples = np.array([
    np.array(test1_samples),
    np.array(test2_samples),
    np.array(test3_samples),
    np.array(test4_samples),
    np.array(test5_samples),
    np.array(test6_samples)
])

df = pd.DataFrame(test_samples, columns=list(test1.result.keys()), index=[
    'eigen', 'between', 'close', 'degree', 'info', 'harmony', 'katz', 'percolation'
]).T
print(df)
df.to_csv('/home/user/Documents/Wilcoxon.csv')

In [84]:
# Engel 1 and 2, global efficiency

datasets = GRAPHS.copy()

for i in range(len(datasets)):
    for sample in datasets[i].index:
        subject = sample[:4]
        if subject in REJECTED:
            datasets[i] = datasets[i].drop(index=sample)
        # if subject not in ENGEL1 and subject not in ENGEL2 or subject in REJECTED:
        #     datasets[i] = datasets[i].drop(index=sample)

In [90]:
# hemispheres

names = ['transitivity', 's_metric', 'global_efficiency']
cross_hemispheres_informativeness_arr = list()
cross_subjects_informativeness_arr = list()

confusions = list()
# tn, fp, fn, tp = confusion_matrix(y_test, pred).ravel()
#     confusions.append((tn, fp, fn, tp))

for dataset, name in zip(datasets, names):
    cross_hemispheres_informativeness = CrossInformativeness()
    cross_subjects_informativeness = CrossInformativeness()

    for _ in range(100):
        hemispheres_informatoveness = Informativeness()
        subjects_informativeness = SubjectsInformativeness()
        acc, spec, sens, pospred, negpred = list(), list(), list(), list(), list()

        for i in range(100):

            y = dataset['resected'].to_numpy()
            x = dataset[[f'{name}_for_wpli_4-8Hz', f'{name}_for_envelope_4-8Hz']].to_numpy()

            scaler = StandardScaler()
            x = scaler.fit_transform(x)

            samples = [[sample] for sample in dataset.index.tolist()]

            x = np.append(x, samples, axis=1)
            engel_1_2_set_x, engel_1_2_set_y = list(), list()
            test_set_x, test_set_y = list(), list()
            for sample_x, sample_y in zip(x, y):
                if any([subject in sample_x[-1] for subject in [*ENGEL1, *ENGEL2]]):
                    engel_1_2_set_x.append(sample_x)
                    engel_1_2_set_y.append(sample_y)
                else:
                    test_set_x.append(sample_x)
                    test_set_y.append(sample_y)

            x_train, x_test_add, y_train, y_test_add = train_test_split(
                engel_1_2_set_x,
                engel_1_2_set_y,
                train_size=0.5
            )
            x_test = test_set_x + x_test_add
            y_test = test_set_y + y_test_add
            # train_samples, test_samples = x_train[:][2], x_test[:][2]
            train_samples = np.array([sample[2] for sample in x_train])
            test_samples = np.array([sample[2] for sample in x_test])
            x_train = np.array([sample[0:2] for sample in x_train])
            x_test = np.array([sample[0:2] for sample in x_test])

            clf = svm.SVC()
            clf.fit(x_train, y_train)
            pred = clf.predict(x_test)

            # if not any([tn + fp == 0, tp + fn == 0, tn + fn == 0, tp + fp == 0]):
            #     spec.append(tn / (tn + fp))
            #     sens.append(tp / (tp + fn))
            #     negpred.append(tn/(tn + fn))
            #     pospred.append(tp/(tp + fp))

            for predicted, actual, sample, value in zip(pred, y_test, test_samples, x_test):
                hemispheres_informatoveness.informativeness = sample, actual, 'correct' \
                if predicted == actual else 'wrong'
                subjects_informativeness.informativeness = sample, actual, 'correct' \
                if predicted == actual else 'wrong'

        cross_subjects_informativeness.informativeness = subjects_informativeness
        cross_hemispheres_informativeness.informativeness = hemispheres_informatoveness

    cross_hemispheres_informativeness_arr.append(cross_hemispheres_informativeness)
    cross_subjects_informativeness_arr.append(cross_subjects_informativeness)

In [91]:
series = list()
index = [
    'Transitivity',
    'S metric',
    'Global Efficiency',
    'Engel'
]

eng = dict()
for subject_name in ENGEL1 + ENGEL2 + ENGEL3 + ENGEL4:
    if subject_name in REJECTED:
        continue
    else:
        if subject_name in ENGEL1:
            eng.update({
                subject_name: 1
            })
        if subject_name in ENGEL2:
            eng.update({
                subject_name: 2
            })
        if subject_name in ENGEL3:
            eng.update({
                subject_name: 3
            })
        if subject_name in ENGEL4:
            eng.update({
                subject_name: 4
            })

for info in cross_subjects_informativeness_arr:
    series.append(
        pd.Series(
                dict(
                sorted(
                    info.acc().items(),
                    key=lambda item: item[0]
                )
            )
        )
    )

series.append(
    pd.Series(
                dict(
                sorted(
                    eng.items(),
                    key=lambda item: item[0]
                )
            )
        )
)

df = pd.DataFrame(
    series,
    index=index
)
df = df.T
print(df)
df.to_csv('~/Documents/Subjects_informativeness-hemispheres-level.csv')

      Transitivity  S metric  Global Efficiency  Engel
B1C2      0.273291  0.355502           0.390818    1.0
B1R1      0.875153  0.877188           0.958836    1.0
G1R1      0.962443  0.968370           0.978824    1.0
G1V2      0.479753  0.490202           0.460029    1.0
J1T2      0.288805  0.584870           0.662457    1.0
K1V1      0.214108  0.527049           0.525609    1.0
K4L2      0.566000  0.558700           0.490300    4.0
L1P1      0.853683  0.950030           0.976848    1.0
L2M1      0.736296  0.887759           0.641966    2.0
M1G2      0.631346  0.727425           0.763141    1.0
M1N2      0.864098  0.832760           0.928898    1.0
M2S2      0.647733  0.695212           0.779264    2.0
N2K2      0.486773  0.523013           0.864838    2.0
N3S2      0.051224  0.040018           0.051564    3.0
O1O2      0.849875  0.918361           0.970726    1.0
R1D2      0.944319  0.860090           0.955945    1.0
S1A2      0.324196  0.392767           0.460512    1.0
S1B1      

In [ ]:
# regions

for stat in stats:
    print(stat)
    start = time.time()
    cross_subjects_informativeness = CrossInformativeness()

    for _ in range(100):
        features = ['4-8Hz_wpli', '4-8Hz_envelope']
        acc, spec, sens, pospred, negpred = list(), list(), list(), list(), list()

        subjects_informativeness = SubjectsInformativeness()

        for _ in range(100):
            clf = svm.SVC()
            true_data = stat.datasets['true'][features]
            false_data = stat.datasets['false_mirror'][features]
            true_data = true_data.assign(resected=True)
            false_data = false_data.assign(resected=False)
            for sample in true_data.index:
                subject = sample[:4]
                # if subject in REJECTED:
                #     true_data = true_data.drop(index=sample)
                if subject not in ENGEL1 and subject not in ENGEL2 or subject in REJECTED:
                    true_data = true_data.drop(index=sample)
            for sample in false_data.index:
                subject = sample[:4]
                # if subject in REJECTED:
                #     false_data = false_data.drop(index=sample)
                if subject not in ENGEL1 and subject not in ENGEL2 or subject in REJECTED:
                    false_data = false_data.drop(index=sample)
            dataset = pd.concat([true_data, false_data], axis=0)
            dataset = dataset.sample(frac = 1)


            scaler = StandardScaler()

            y = dataset['resected'].to_numpy()
            dataset = dataset.drop(['resected'], axis=1)
            samples = [[sample] for sample in dataset.index.tolist()]

            x = scaler.fit_transform(dataset)
            x = np.append(x, samples, axis=1)
            x_train, x_test, y_train, y_test = train_test_split(x, y)
            train_samples, test_samples = x_train[:, 2], x_test[:, 2]
            x_train, x_test = x_train[:, 0:2], x_test[:, 0:2]

            clf.fit(x_train, y_train)
            pred = clf.predict(x_test)

            for predicted, actual, sample, value in zip(pred, y_test, test_samples, x_test):
                subjects_informativeness.informativeness = sample, actual, 'correct'\
                    if predicted == actual else 'wrong'
            # acc.append(accuracy_score(y_test, pred))

            # if not any([tn + fp == 0, tp + fn == 0, tn + fn == 0, tp + fp == 0]):
            #     spec.append(tn / (tn + fp))
            #     sens.append(tp / (tp + fn))
            #     negpred.append(tn/(tn + fn))
            #     pospred.append(tp/(tp + fp))
        cross_subjects_informativeness.informativeness = subjects_informativeness

    print(f'RUNTIME: {time.time() - start}')
    cross_subjects_informativeness_arr.append(cross_subjects_informativeness)

In [ ]:
series = list()
index = [
    'Transitivity',
    'S metric',
    'Global Efficiency',
    'Eigencentrality',
    'Betweenness',
    'Closeness',
    'Degree',
    'Info',
    'Harmony',
    'Katz',
    'Percolation',
    'Engel'
]

eng = dict()
for subject_name in ENGEL1 + ENGEL2:
    if subject_name in REJECTED:
        continue
    else:
        eng.update({
            subject_name: 1 if subject_name in ENGEL1 else 2
        })

for info in cross_subjects_informativeness_arr:
    series.append(
        pd.Series(
                dict(
                sorted(
                    info.acc().items(),
                    key=lambda item: item[0]
                )
            )
        )
    )

series.append(
    pd.Series(
                dict(
                sorted(
                    eng.items(),
                    key=lambda item: item[0]
                )
            )
        )
)

df = pd.DataFrame(
    series,
    index=index
)
df = df.T
print(df)
df.to_csv('~/Documents/Subjects_informativeness-Engel1&2.csv')

In [ ]:
# hemispheres

names = ['transitivity', 's_metric', 'global_efficiency']
n_iter = 1000
confusions = list()
#     confusions.append((tn, fp, fn, tp))

for dataset, name in zip(datasets, names):

    tn_c, tp_c, fn_c, fp_c = 0, 0, 0, 0
    for i in range(n_iter):

        y = dataset['resected'].to_numpy()
        x = dataset[[f'{name}_for_wpli_4-8Hz', f'{name}_for_envelope_4-8Hz']].to_numpy()

        scaler = StandardScaler()
        x = scaler.fit_transform(x)

        samples = [[sample] for sample in dataset.index.tolist()]

        x = np.append(x, samples, axis=1)
        x_train, x_test, y_train, y_test = train_test_split(x, y)
        train_samples, test_samples = x_train[:, 2], x_test[:, 2]
        x_train, x_test = x_train[:, 0:2], x_test[:, 0:2]

        clf = svm.SVC()
        clf.fit(x_train, y_train)
        pred = clf.predict(x_test)
        tn, fp, fn, tp = confusion_matrix(y_test, pred, labels=[0,1]).ravel()
        tn_c += tn
        fp_c += fp
        fn_c += fn
        tp_c += tp

    confusions.append((tn_c/n_iter, fp_c/n_iter, fn_c/n_iter, tp_c/n_iter))

In [ ]:
# regions

n_iter = 1000

for stat in stats:

    features = ['4-8Hz_wpli', '4-8Hz_envelope']
    tn_c, tp_c, fn_c, fp_c = 0, 0, 0, 0

    for _ in range(n_iter):
        clf = svm.SVC()
        true_data = stat.datasets['true'][features]
        false_data = stat.datasets['false_mirror'][features]
        true_data = true_data.assign(resected=True)
        false_data = false_data.assign(resected=False)
        for sample in true_data.index:
            subject = sample[:4]
            if subject in REJECTED:
                true_data = true_data.drop(index=sample)
        for sample in false_data.index:
            subject = sample[:4]
            if subject in REJECTED:
                false_data = false_data.drop(index=sample)
        dataset = pd.concat([true_data, false_data], axis=0)
        dataset = dataset.sample(frac = 1)


        scaler = StandardScaler()

        y = dataset['resected'].to_numpy()
        dataset = dataset.drop(['resected'], axis=1)
        samples = [[sample] for sample in dataset.index.tolist()]

        x = scaler.fit_transform(dataset)
        x = np.append(x, samples, axis=1)
        x_train, x_test, y_train, y_test = train_test_split(x, y)
        train_samples, test_samples = x_train[:, 2], x_test[:, 2]
        x_train, x_test = x_train[:, 0:2], x_test[:, 0:2]

        clf.fit(x_train, y_train)
        pred = clf.predict(x_test)

        tn, fp, fn, tp = confusion_matrix(y_test, pred, labels=[0,1]).ravel()
        tn_c += tn
        fp_c += fp
        fn_c += fn
        tp_c += tp

    confusions.append((tn_c/n_iter, fp_c/n_iter, fn_c/n_iter, tp_c/n_iter))

In [ ]:
index = [
    'Transitivity',
    'S metric',
    'Global Efficiency',
    'Eigencentrality',
    'Betweenness',
    'Closeness',
    'Degree',
    'Info',
    'Harmony',
    'Katz',
    'Percolation'
]

columns = [
    'accuracy',
    'specificity',
    'sensitivity',
    'Matthews',
    'f1',
    'Fowlkes–Mallows'
]

series = list()

for tn, fp, fn, tp in confusions:

    series.append(
        [
            (tp + tn) / (tp + tn + fp + fn),
            tn / (tn + fp),
            tp / (tp + fn),
            (tp * tn - fp * fn) /
            np.sqrt(
                (tp + fp) *
                (tp + fn) *
                (tn + fp) *
                (tn + fn)
            ),
            2 * tp / (2 * tp + fp + fn),
            np.sqrt(tp / (tp + fn) * tp / (tp + fp))
        ]
    )

df = pd.DataFrame(
    series,
    columns=columns,
    index=index
)
df = df.T
# print(df)
df.to_csv('~/Documents/Metrics_informativeness.csv')


In [ ]:
for name, info in zip(names, cross_subjects_informativeness_arr):

    acc = dict(
        sorted(
            info.acc().items(),
            key=lambda item: item[0]
        )
    )

    plt.bar(
        acc.keys(),
        acc.values()
    )
    plt.xticks(rotation=90)
    # plt.xticks(range(len(info.acc().keys())))
    plt.title(f'Subjects-level prediction accuracy, {name}')
    plt.show()


In [ ]:
series = list()
names = ['Transitivity, acc, %', 'S metric, acc, %', 'Global Efficiency, acc, %', 'Eigencentrality, acc, %']
for info in cross_subjects_informativeness_arr:
    series.append(
        pd.Series(
            dict(
                sorted(
                    info.acc().items(),
                    key=lambda item: item[0]
                )
            )
        )
    )


df = pd.DataFrame(series, index=names)

df = df.T
print(df)
df.to_csv('/home/user/Documents/Engel1&2_Subject_informativeness.csv')